In [5]:
from daq import daqDevice
import daqh
import numpy as np
import time
import sys
import os
sys.path.append('C:\\software_git_repos\\cofe-python-analysis-tools\\utils_meinhold')
sys.path.append('C:\\software_git_repos\\cofe-python-analysis-tools\\utils_zonca')
sys.path.append('C:\\software_git_repos\\cofe-python-analysis-tools\\utils_zonca\\demod')
sys.path.append('C:\\Anaconda3\\envs\\py27\\Scripts')
import h5py
%pylab

import prm_util as cu

from Tkinter import Tk
from tkFileDialog import askopenfilename

acqmode=daqh.DaamInfinitePost

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [6]:
def get_data(nchan=4,freq=10,nseconds=5,comment='None',alerts=[58,59,60,118,119,120,178,179,180,238,239,240,298,299,300]):
    """
    function to simply aquire nchan a/d channels at rate freq
    for nseconds seconds
    """
    
    #outdata=np.zeros([nchan,nscans],dtype=float)
    dev=daqDevice('DaqBoard3001USB')
    gains=[]
    flags=[]
    chans=[]
    if nchan > 8:
        uchan=nchan-8
        for i in range(8):
            gains.append(daqh.DgainX1)
            flags.append(daqh.DafBipolar|daqh.DafDifferential)
            chans.append(i)
        for i in range(uchan):
            gains.append(daqh.DgainX1)
            flags.append(daqh.DafBipolar|daqh.DafDifferential)
            chans.append(256+i)   #HERE is the famous fix where DaqX refs upper level dif channels!
    elif nchan<9:      
        for i in range(nchan):
            gains.append(daqh.DgainX1)
            flags.append(daqh.DafBipolar|daqh.DafDifferential)
            chans.append(i)
    acqmode = daqh.DaamNShot
    dev.AdcSetAcq(acqmode, postTrigCount = nseconds*freq)
    dev.AdcSetScan(chans,gains,flags)
    dev.AdcSetFreq(freq)
    #use the driver buffer here user buffer was very limited (the way I tried anyway) 
    transMask = daqh.DatmUpdateBlock|daqh.DatmCycleOn|daqh.DatmDriverBuf

    buf=dev.AdcTransferSetBuffer(transMask, np.uint(nseconds*freq*nchan))
    #bufMask is for transferring the buffer
    bufMask = daqh.DabtmOldest | daqh.DabtmRetAvail

    timestart = (time.time())
    timenotify = timestart + 5

    dev.SetTriggerEvent(daqh.DatsImmediate,None, 0, np.array(gains[0],dtype=int), np.array(flags[0],dtype=int), daqh.DaqTypeAnalogLocal, 0, 0, daqh.DaqStartEvent)
    dev.SetTriggerEvent(daqh.DatsScanCount,None, 0, np.array(gains[0],dtype=int), np.array(flags[0],dtype=int), daqh.DaqTypeAnalogLocal, 0, 0, daqh.DaqStopEvent)
    dev.AdcTransferStart()
    dev.AdcArm()
    
    while True:
        alertscopy=alerts[:]
        timenotify = checkAlerts(timenotify, timestart, alerts,alertscopy)        
        
        stat = dev.AdcTransferGetStat()
        active = stat['active']
        if not (active & daqh.DaafAcqActive):
            break
    dev.AdcDisarm()
    outdata,ret=dev.AdcTransferBufData(nseconds*freq, nchan,bufMask)
    
    outdata=np.array(outdata,dtype=float)
    outdata=(outdata-2**15)*20./2**16
    outdata=np.transpose(np.reshape(outdata,[nseconds*freq,nchan]))
    print "Finished collecting data\n-"
    dev.Close()    
    return outdata

In [7]:
def import_data(filename=None):
    '''
    function to read in previously stored data and put in format of freshly read data dictionary
    '''
    
    if filename==None:
        Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
        filename = askopenfilename(initialdir='C:\\Interferometer_tests\\io_3001_usb_data')
    hf=h5py.File(filename)
    d={}
    print(hf.keys())
    for key in hf.keys():
        d[key]=squeeze(array(hf[key]))
    hf.close()
    return d

In [8]:
def checkAlerts(timeupdate,timestart, alerts, alertscopy, updateincrement=5):
    timecheck = (time.time())
    if timecheck > timeupdate:
        print "Time: " + str(int(timecheck - timestart))
        timeupdate += updateincrement
    for alert in alerts:
        if (timecheck - timestart)%60 > alert:
            print "---- " + str(int(timecheck - timestart)) + " SECONDS ----"
            alerts.remove(alert)
    if 2<(timecheck -timestart%60)<3:
        alerts = alertscopy[:]
        
        
    return timeupdate

In [9]:
def get_date_filename(labelstring=''):
    #version for testing adds an optional string to identify tests
    now=time.localtime()[0:6]
    #dirfmt = "c:\\cofe\\ground_data\\testdata\\%4d_%02d_%02d"
    dirfmt = "C:\\Interferometer_tests\\io_3001_usb_data\\%4d_%02d_%02d"
    dirname = dirfmt % now[0:3]
    filefmt = "%02d_%02d_%02d.h5"
    filename= labelstring+filefmt % now[3:6]
    ffilename=os.path.join(dirname,filename)
    if not os.path.exists(dirname):
        os.mkdir(dirname)
    return(ffilename)

In [10]:
def zplot(d_dict_list,minfreq=1):
    """
    calculate asd's and overplot all from list of data dictionaries, assumed to have meaningful
    keynames for legend
    """
    figure()
    for d_dict in d_dict_list:
        z=cu.nps(d_dict['data'],d_dict['samprate'],minfreq=minfreq)
        plot(z[0],np.sqrt(z[1]),label=d_dict['label'])
    xlabel('Frequency, [Hz]')
    ylabel('ASD, [$nV \sqrt{sec}$]')
    xscale('log')
    yscale('log')
    legend()
    title('Comparison spectra')
    legend()

In [11]:
#use this function- box is set up with terminations on alternate channels to increase isolation
def get_test(samprate=10000,nseconds=10,labelstring='test',minfreq=1):
    s=[0]  #active channels, intervening ones are terminated to increase isolation
    d=get_data(nchan=1,freq=samprate,nseconds=nseconds)
    #z1=cu.nps(d[0,:],samprate,minfreq)
    
    #figure()
    #plot(z1[0],np.sqrt(z1[1]),label='z1')
    #ylim([0,.05])
    #xlabel('Frequency, [Hz]')
    #label('ASD, [$V \sqrt{sec}$]')
    #title('Test')
    #legend()
    #yscale('log')
    #xscale('log')
    outdata=squeeze(d[s,:])
    fname=get_date_filename(labelstring)
    with h5py.File(fname, mode='w') as hdf_file:
        hdf_file.create_dataset('data', data=outdata)
        hdf_file.create_dataset('samprate', data=[samprate])
        hdf_file.create_dataset('label', data=[labelstring])
        hdf_file.close()
    return {'data':outdata,'samprate':samprate,'label':labelstring}

In [12]:
#use this function- box is set up with terminations on alternate channels to increase isolation
def get_test2(samprate=10000,nseconds=10,labelstring='test',minfreq=1):
    s=[0,2]  #active channels, intervening ones are terminated to increase isolation
    d=get_data(nchan=4,freq=samprate,nseconds=nseconds)
    #z1=cu.nps(d[0,:],samprate,minfreq)
    
    #figure()
    #plot(z1[0],np.sqrt(z1[1]),label='z1')
    #ylim([0,.05])
    #xlabel('Frequency, [Hz]')
    #label('ASD, [$V \sqrt{sec}$]')
    #title('Test')
    #legend()
    #yscale('log')
    #xscale('log')
    outdata=squeeze(d[s,:])
    fname=get_date_filename(labelstring)
    with h5py.File(fname, mode='w') as hdf_file:
        hdf_file.create_dataset('data', data=outdata)
        hdf_file.create_dataset('samprate', data=[samprate])
        hdf_file.create_dataset('label', data=[labelstring])
        hdf_file.close()
    return {'data':outdata,'samprate':samprate,'label':labelstring}

In [13]:
def demod(indata):
    #function to extract I,Q from data, sync
    dd=indata['data'][1,4:] - indata['data'][1,:-4]
    signal=indata['data'][0,4:]
    x=arange(len(dd))
    d_edges=x[where(dd > 2.)]
    u_edges=x[where(dd < -2.)]
    #trim these lists to avoid partials at the end, and force first edge to be rising 
    #last edge to be falling one
    u_edges=u_edges[1:-1]
    d_edges=d_edges[where(d_edges > u_edges[0])]
    d_edges=d_edges[where(d_edges < u_edges[-1])]
    phase1=[]
    phase2=[]
    print(len(d_edges),len(u_edges))
    for u,d,u2 in zip(u_edges[:-1],d_edges,u_edges[1:]):
        if(((d-u)>5) and ((d-u) <100) and ((u2-d>5)) and ((u2-d) <100)):
            phase1.append(np.mean(signal[u:d]))
            phase2.append(np.mean(signal[d:u2]))
    i=np.array(phase1)
    q=np.array(phase2)
    z=i-mean(i) + 1j*(q-mean(q))
    phi=np.unwrap(np.angle(z))
    return i,q,phi
    #return x,u_edges,d_edges,dd

In [14]:
d=get_test2(nseconds=10,samprate=20000,labelstring='tst')

Time: 5
Time: 10
Finished collecting data
-


WindowsError: [Error 3] The system cannot find the path specified: 'C:\\Interferometer_tests\\io_3001_usb_data\\2016_12_22'

In [22]:
i6,q6,phi6=demod(d6)

(19996, 19998)


In [23]:
figure(3)

plot(phi6)

In [24]:
figure()
plot(i6)
plot(q6)

In [243]:
figure()
plot(d7['data'][0,:])

In [191]:
z=i+1j*q
phi=np.angle(z)
uphi=np.unwrap(phi,discont=-np.pi/4)
figure()
plot(uphi)

In [266]:
figure()
plot(d1['data'][1,:1000])

In [169]:
figure()
plot(phi)


In [155]:
print(i[:10])

[]


In [47]:
zplot([int2d],minfreq=.01)

In [44]:
int2d2['data']

array([[ 0.15289307,  0.15197754,  0.15197754, ...,  0.1574707 ,
         0.15686035,  0.15716553],
       [ 0.19012451,  0.18981934,  0.19104004, ...,  0.18463135,
         0.18493652,  0.18432617]])

In [45]:
figure()
plot(int2d2['data'][0])
plot(int2d2['data'][1])

In [25]:
int2d2['data']

array([ 0.12939453,  0.12939453,  0.13000488, ...,  0.13580322,
        0.1361084 ,  0.13580322])

In [25]:
figure()
plot(ton1['data'])

In [105]:
zplot([tf,t])

In [170]:
x=arange(1e4)
i=np.sin(x*np.pi/1000.)
q=np.cos(x*np.pi/1000.)
z=i+1j*q

In [29]:
z[0]

1j

In [30]:
figure()
plot(z)

In [33]:
plot(np.angle(z))

In [171]:
figure()
plot(x,i)
plot(x,q)
plot(x,np.angle(z))
plot(x,np.unwrap(np.angle(z)))

In [40]:
figure()
plot(x,i)
plot(x,q)